Шифр Цезаря – это один из шифров подстановки, в котором каждый символ открытого текста заменяется символом, находящимся на некотором, определяемом ключом, постоянном числе позиций левее или правее него в алфавите.

Для реализация, создадим строку, вклдчающую в себя все символы русского алфавита:

In [412]:
alphabet = "АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЭЮЯ"

print(f"Алфавит без сдвига: `{alphabet}`")

Алфавит без сдвига: `АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЭЮЯ`


Теперь, необходимо создать зашифрованный алфавит. Для этого создадим функцию, смещающую алфавит на необходимое количество символов:

In [413]:
def encrypt_alphabet(alphabet: str, shift: int) -> str:
    cipher_alphabet = ''

    for char in alphabet:
        shifted_index = (alphabet.index(char) + shift) % len(alphabet)
        cipher_char = alphabet[shifted_index]
        cipher_alphabet += cipher_char

    return cipher_alphabet

encrypted_alphabet = encrypt_alphabet(alphabet=alphabet, shift=3)

print(f"Алфавит со сдвигом: `{encrypted_alphabet}`")

Алфавит со сдвигом: `ГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЭЮЯАБВ`


Далее нам необходимо реализовать функционал шифрования данного текста, для этого мы создадим класс, который будет инкапсулировать весь функционал, связанный с шифрованием:

In [414]:
class CaesarEncryptor:

    def __init__(self, alphabet: str, alphabet_encrypter, shift: int) -> None:
        self._alphabet = alphabet
        self._encrypted_alphabet = alphabet_encrypter(alphabet, shift)

    def apply(self, string: str) -> str:
        string = string.upper()
        result = ""
        for char in string:
            if char in alphabet:
                result += self._encrypted_alphabet[self._alphabet.find(char)]
            else:
                result += char
        return result

Опробуем созданный шифратор на нескольких примерах, для этого создадим объект шифратора, используя ранее созданную функцию шифра алфавита и сдвиг в 3 буквы вправо:

In [415]:
key = CaesarEncryptor(alphabet=alphabet, alphabet_encrypter=encrypt_alphabet, shift=3)

t1 = "Тише едешь - дальше будешь"
t2 = "Жизнь пройти - не поле перейти"
t3 = "Семь раз отмерь - один раз отрежь"

e1 = key.apply(t1)
e2 = key.apply(t2)
e3 = key.apply(t3)

from IPython.display import HTML, display
import pandas as pd

df = pd.DataFrame([
    [t1, e1], [t2, e2], [t3, e3]
], columns=["Исходный текст", "Зашифрованный текст"])

display(HTML(df.to_html(index=None)))

Исходный текст,Зашифрованный текст
Тише едешь - дальше будешь,ХЛЫЗ ЗЖЗЫЬ - ЖГОЬЫЗ ДЦЖЗЫЬ
Жизнь пройти - не поле перейти,ЙЛКРЬ ТУСМХЛ - РЗ ТСОЗ ТЗУЗМХЛ
Семь раз отмерь - один раз отрежь,ФЗПЬ УГК СХПЗУЬ - СЖЛР УГК СХУЗЙЬ


Для того чтобы проверить корректность работы шифратора, необходимо лишь создать ещё один, со сдвигом влево на 3 символа, который будет действовать как расшифровщик (расшифрование - процесс преобразования зашифрованного сообщения в открытый текст, когда вы знаете ключ шифрования):

In [416]:
decryptor = CaesarEncryptor(alphabet=alphabet, alphabet_encrypter=encrypt_alphabet, shift=-3)

d1 = decryptor.apply(e1)
d2 = decryptor.apply(e2)
d3 = decryptor.apply(e3)

df = pd.DataFrame([
    [e1, d1], [e2, d2], [e3, d3]
], columns=["Зашифрованный текст", "Расшифрованный текст"])

display(HTML(df.to_html()))


,Зашифрованный текст,Расшифрованный текст
0,ХЛЫЗ ЗЖЗЫЬ - ЖГОЬЫЗ ДЦЖЗЫЬ,ТИШЕ ЕДЕШЬ - ДАЛЬШЕ БУДЕШЬ
1,ЙЛКРЬ ТУСМХЛ - РЗ ТСОЗ ТЗУЗМХЛ,ЖИЗНЬ ПРОЙТИ - НЕ ПОЛЕ ПЕРЕЙТИ
2,ФЗПЬ УГК СХПЗУЬ - СЖЛР УГК СХУЗЙЬ,СЕМЬ РАЗ ОТМЕРЬ - ОДИН РАЗ ОТРЕЖЬ


Таким образом, зная, что для шифрования текста был использован шифр Цезаря, его дешифрование (взлом шифра, попытка вскрытия сообщения, когда вы НЕ знаете ключ шифрования) не представляет сложности, необходимо лишь перебрать небольшое количество возможных сдвигов.

Попробуем таким образом взломать первый шифр:

In [417]:
possible_original_texts = []

for shift in range(len(alphabet)):
    possible_decryptor = CaesarEncryptor(alphabet=alphabet, alphabet_encrypter=encrypt_alphabet, shift=shift)
    possible_original_texts.append([shift, possible_decryptor.apply(e1)])

df = pd.DataFrame(possible_original_texts, columns=["Сдвиг", "Текст"])
display(HTML(df.to_html(index=None)))

Сдвиг,Текст
0,ХЛЫЗ ЗЖЗЫЬ - ЖГОЬЫЗ ДЦЖЗЫЬ
1,ЦМЭИ ИЗИЭЬ - ЗДПЬЭИ ЕЧЗИЭЬ
2,ЧНЮЙ ЙИЙЮЬ - ИЕРЬЮЙ ЁШИЙЮЬ
3,ШОЯК КЙКЯЬ - ЙЁСЬЯК ЖЩЙКЯЬ
4,ЩПАЛ ЛКЛАЬ - КЖТЬАЛ ЗЪКЛАЬ
5,ЪРБМ МЛМБЬ - ЛЗУЬБМ ИЫЛМБЬ
6,ЫСВН НМНВЬ - МИФЬВН ЙЭМНВЬ
7,ЭТГО ОНОГЬ - НЙХЬГО КЮНОГЬ
8,ЮУДП ПОПДЬ - ОКЦЬДП ЛЯОПДЬ
9,ЯФЕР РПРЕЬ - ПЛЧЬЕР МАПРЕЬ


Таким образом, можно сделать вывод, что в современных реалиях шифр Цезаря невероятно слаб. Во времена жизни Цезаря, ставка делалась на то, что потенциальному взломщику незнаком алгоритм шифрования и уж тем более основы криптографии, что было вполне ожидаемо при уровне образования и развития криптографии того времени.

В наши дни, делать ставку на незнание противником алгоритма нельзя. Также, с учетом доступных ныне вычислительных мощностей, 33 варианта ключа - невероятно маленькое количество, которое любой компьютер способен проверить за доли секунды, именно поэтому шифр Цезаря нельзя считать безопасным.

Иной слабостью является однозначность соответствия букв - одна и та же буква оригинального текста всегда соответствует одной и той же букве зашифрованного текста.

Попробуем усложнить шифр, решив две вышеописанные проблемы: в качестве ключа мы будем использовать некоторую математическую функцию, которая для каждого отдельного символа будет определять сдвиг на основании его положения в тексте. Таким образом, для расшифрования текста будет необходимо знать эту самую функцию, число которых запредельно, что не позволит взломать шифр перебором, а использование положения символа в тексте позволит избежать взлома путём частотного анализа:

In [418]:
class BetterCaesarEncryptor:

    def __init__(self, alphabet: str, alphabet_encrypter, shift_supplier) -> None:
        self._alphabet = alphabet
        self._alphabet_encrypter = alphabet_encrypter
        self._shift_supplier = shift_supplier

    def apply(self, string: str) -> str:
        string = string.upper()
        result = ""
        for pos, char in enumerate(string):
            if char in alphabet:
                encrypted_alphabet = self._alphabet_encrypter(alphabet, self._shift_supplier(pos))
                result += encrypted_alphabet[self._alphabet.find(char)]
            else:
                result += char
        return result

В качестве примера, используем такую функцию:

In [419]:
def key_function(pos: int) -> int:
    return 17 * pos - 518 

Создадим шифратор и зашифруем несколько текстов:

In [420]:
encryptor = BetterCaesarEncryptor(
    alphabet=alphabet,
    alphabet_encrypter=encrypt_alphabet,
    shift_supplier=key_function
)

e1 = encryptor.apply(t1)
e2 = encryptor.apply(t2)
e3 = encryptor.apply(t3)

df = pd.DataFrame([
    [t1, e1], [t2, e2], [t3, e3]
], columns=["Оригинальный текст", "Зашифрованный текст"], index=None)

display(HTML(df.to_html()))

,Оригинальный текст,Зашифрованный текст
0,Тише едешь - дальше будешь,МУФС УДХЪЬ - ЪЗЕЬУР ОТУЁКЬ
1,Жизнь пройти - не поле перейти,БУДЪЬ ПВРЭЦЮ - ЖО ЫЛЩД РЦУШОИП
2,Семь раз отмерь - один раз отрежь,ЛПИЬ ААШ ВЦВКЗЬ - ЪБЦМ ССК УИЧЭПЬ


Для того, чтобы расшифровать сообщения, понадобится использовать такой же шифратор, но с отраженным сдвигом:

In [421]:
def reverse_key(pos: int):
    return -key_function(pos)

decryptor = BetterCaesarEncryptor(
    alphabet=alphabet,
    alphabet_encrypter=encrypt_alphabet,
    shift_supplier=reverse_key
)

d1 = decryptor.apply(e1)
d2 = decryptor.apply(e2)
d3 = decryptor.apply(e3)

df = pd.DataFrame([
    [e1, d1], [e2, d2], [e3, d3]
], columns=["Зашифрованный текст", "Расшифровнный текст"], index=None)

display(HTML(df.to_html()))

,Зашифрованный текст,Расшифровнный текст
0,МУФС УДХЪЬ - ЪЗЕЬУР ОТУЁКЬ,ТИШЕ ЕДЕШЬ - ДАЛЬШЕ БУДЕШЬ
1,БУДЪЬ ПВРЭЦЮ - ЖО ЫЛЩД РЦУШОИП,ЖИЗНЬ ПРОЙТИ - НЕ ПОЛЕ ПЕРЕЙТИ
2,ЛПИЬ ААШ ВЦВКЗЬ - ЪБЦМ ССК УИЧЭПЬ,СЕМЬ РАЗ ОТМЕРЬ - ОДИН РАЗ ОТРЕЖЬ
